In [1]:
import os
SPARK_HOME="/opt/spark"
os.environ["PYSPARK_PYTHON"] = "python3"
# os.environ["PYSPARK_PYTHON"]="python"
os.environ["PYSPARK_DRIVER_PYTHON"]="jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook"
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["SPARK_CLASSPATH"] = "/opt/spark/jars/sqljdbc4.jar"

 

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("new") \
        .getOrCreate()

In [2]:
#from pyspark.sql import SQLContext
# sqlContext = SQLContext(spark)
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
df_path = "/home/rupeshr/Documents/split_Test.csv"
df =spark.read.format("com.databricks.spark.csv")\
                    .option("multiline", "true")\
                    .option("header", "true")\
                    .option("inferSchema", "true")\
                    .option("sep", ',')\
                    .option('escape', '\"')\
                    .option("allowSingleQuotes", "true")\
                    .option("ignoreLeadingWhiteSpace", "true")\
                    .option("ignoreTrailingWhiteSpace", "true")\
                    .load(df_path)
df.printSchema()
df.show()

root
 |-- column: string (nullable = true)
 |-- name: string (nullable = true)

+------+----+
|column|name|
+------+----+
|    12| 123|
|    14| 132|
|   abc|said|
|   efg| we3|
|     2| sa1|
|   qwe| as2|
| 12ews|  34|
|   aws| saw|
|    34|said|
|    56|hold|
|  said|  12|
+------+----+



In [6]:
def is_digit(value):
    if value:
        return value.isdigit()
    else:
        return False

is_digit_udf = udf(is_digit, BooleanType())

df = df.withColumn('numerical', when(is_digit_udf(df['name']), df['name']))
df = df.withColumn('categorical', when(~is_digit_udf(df['name']), df['name']))
df.show()

+------+----+---------+-----------+
|column|name|numerical|categorical|
+------+----+---------+-----------+
|    12| 123|      123|       null|
|    14| 132|      132|       null|
|   abc|said|     null|       said|
|   efg| we3|     null|        we3|
|     2| sa1|     null|        sa1|
|   qwe| as2|     null|        as2|
| 12ews|  34|       34|       null|
|   aws| saw|     null|        saw|
|    34|said|     null|       said|
|    56|hold|     null|       hold|
|  said|  12|       12|       null|
+------+----+---------+-----------+

